In [1]:
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
from datasets import Dataset
import data
from os.path import join as oj
import pickle as pkl
import os
import utils

# Baseline results
These are the results for when we just query the model to ask it the relationship after few-shot prompting with a single example.

In [ ]:
results_dir = '/home/chansingh/mntv1/single_query'

In [60]:
dir_names = sorted([fname
                    for fname in os.listdir(results_dir)
                    if os.path.isdir(oj(results_dir, fname))
                    and os.path.exists(oj(results_dir, fname, 'results_final.pkl'))
                    ])
results_list = []
for dir_name in tqdm(dir_names):
    try:
        ser = pd.Series(pkl.load(open(oj(results_dir, dir_name, 'results_final.pkl'), "rb")))
        ser_scalar = ser[~ser.apply(lambda x: isinstance(x, list)).values.astype(bool)] # only keep scalar-valued vars
        results_list.append(ser_scalar)
    except:
        print('skipping', dir_name)

r = pd.concat(results_list, axis=1).T.infer_objects()
r.to_pickle(os.path.join(results_dir, 'r.pkl'))

100%|██████████| 115/115 [01:17<00:00,  1.49it/s]


In [59]:
r = pd.read_pickle(os.path.join(results_dir, 'r.pkl'))

# drop some things to make it easier to see
cols = r.columns
cols_to_drop = [k for k in cols
                if k.endswith('prefix') or k.endswith('init')]
cols_to_drop += ['epoch_save_interval', 'batch_size']
r = r.drop(columns=cols_to_drop)

KeyError: "['epoch_save_interval', 'batch_size'] not found in axis"

In [49]:
r['final_answer_found'] = (~r['final_answer_full'].isna()).astype(int)

In [50]:
r.groupby(['checkpoint', 'n_shots']).mean().round(2) # mean over templates, task_name

max_digit  template_num_init_string  \
checkpoint  n_shots                                        
gpt2-large  1             10.0                      0.50   
            5             10.0                      0.44   
gpt2-medium 1             10.0                      0.50   
            5             10.0                      0.47   
gpt2-xl     1             10.0                      0.50   
            5             10.0                      0.44   

                     template_num_task_phrasing  max_num_tokens  \
checkpoint  n_shots                                               
gpt2-large  1                              0.50             4.0   
            5                              0.50             4.0   
gpt2-medium 1                              0.50             4.0   
            5                              0.47             4.0   
gpt2-xl     1                              0.50             4.0   
            5                              0.50             4.0   

                     beam_width_suffix  single_query  seed  \
checkpoint  n_shots                                          
gpt2-large  1                      5.0           1.0   1.0   
            5                      5.0           1.0   1.0   
gpt2-medium 1                      5.0           1.0   1.0   
            5                      5.0           1.0   1.0   
gpt2-xl     1                      5.0           1.0   1.0   
            5                      5.0           1.0   1.0   

                     final_model_queries  final_num_suffixes_checked  \
checkpoint  n_shots                                                    
gpt2-large  1                       1.00                        2.44   
            5                       1.00                        2.44   
gpt2-medium 1                       4.81                        4.12   
            5                       1.00                        2.56   
gpt2-xl     1                       2.00                        3.69   
            5                       2.94                        5.88   

                     final_answer_found  
checkpoint  n_shots                      
gpt2-large  1                      0.80  
            5                      0.89  
gpt2-medium 1                      0.80  
            5                      0.84  
gpt2-xl     1                      0.80  
            5                      0.89

In [ ]:
r.groupby(['checkpoint', 'n_shots', 'task_name']).mean().round(2) # mean over templates

max_digit  template_num_init_string  \
checkpoint  n_shots task_name                                           
gpt2-large  1       add_two            10.0                       0.5   
                    divide_two         10.0                       0.5   
                    max_two            10.0                       0.5   
                    multiply_two       10.0                       0.5   
                    subtract_two       10.0                       0.5   
            5       add_two            10.0                       0.5   
                    divide_two         10.0                       0.5   
                    max_two            10.0                       0.0   
                    multiply_two       10.0                       0.5   
                    subtract_two       10.0                       0.5   
gpt2-medium 1       add_two            10.0                       0.5   
                    divide_two         10.0                       0.5   
                    max_two            10.0                       0.5   
                    multiply_two       10.0                       0.5   
                    subtract_two       10.0                       0.5   
            5       add_two            10.0                       0.5   
                    divide_two         10.0                       0.5   
                    max_two            10.0                       0.0   
                    multiply_two       10.0                       0.5   
                    subtract_two       10.0                       0.5   
gpt2-xl     1       add_two            10.0                       0.5   
                    divide_two         10.0                       0.5   
                    max_two            10.0                       0.5   
                    multiply_two       10.0                       0.5   
                    subtract_two       10.0                       0.5   
            5       add_two            10.0                       0.5   
                    divide_two         10.0                       0.5   
                    max_two            10.0                       0.0   
                    multiply_two       10.0                       0.5   
                    subtract_two       10.0                       0.5   

                                  template_num_task_phrasing  max_num_tokens  \
checkpoint  n_shots task_name                                                  
gpt2-large  1       add_two                              0.5             4.0   
                    divide_two                           0.5             4.0   
                    max_two                              0.5             4.0   
                    multiply_two                         0.5             4.0   
                    subtract_two                         0.5             4.0   
            5       add_two                              0.5             4.0   
                    divide_two                           0.5             4.0   
                    max_two                              0.5             4.0   
                    multiply_two                         0.5             4.0   
                    subtract_two                         0.5             4.0   
gpt2-medium 1       add_two                              0.5             4.0   
                    divide_two                           0.5             4.0   
                    max_two                              0.5             4.0   
                    multiply_two                         0.5             4.0   
                    subtract_two                         0.5             4.0   
            5       add_two                              0.5             4.0   
                    divide_two                           0.5             4.0   
                    max_two                              0.5             4.0   
                    multiply_two                         0.5             4.0   
                 

# not currently working

**load many results**

In [ ]:
results_dir = '/home/chansingh/mntv1/sweep2'
dir_names = sorted([fname
                    for fname in os.listdir(results_dir)
                    if os.path.isdir(oj(results_dir, fname))
                    ])
results_list = [pd.Series(pkl.load(open(oj(results_dir, dir_name, 'results_final.pkl'), "rb")))
                for dir_name in tqdm(dir_names)]
r = pd.concat(results_list, axis=1).T.infer_objects() #.drop(columns='model')



100%|██████████| 16/16 [00:00<00:00, 166.98it/s]


**look at individual result**

In [51]:
results_dir = '/home/chansingh/mntv1/sweep0/Sep_02_21_24_zjpbgvrtlohx'
r = pkl.load(open(oj(results_dir, 'results_final.pkl'), 'rb'))

FileNotFoundError: [Errno 2] No such file or directory: '/home/chansingh/mntv1/sweep0/Sep_02_21_24_zjpbgvrtlohx/results_final.pkl'

In [4]:
pd.DataFrame.from_dict(r)

,max_digit,n_shots,task_name,checkpoint,prefix_or_suffix,max_num_tokens,lr_prefix,beam_width_suffix,batch_size,seed,...,suffix_str_init,len_suffix_str_init,suffix_str_full,suffix_str_added,correct,num_model_queries,decoded_token,top_decoded_tokens_dict,final_answer_full,final_answer_added
0,10,1,add_two,gpt2-medium,suffix,4,0.0001,5,100,1,...,"To get the answer, take the two inputs and",42,"To get the answer, take the two inputs and",,False,1,multiply,"{' multiply': 0.0, ' divide': -0.36162108, ' a...","To get the answer, take the two inputs and add",add


# suffix example results

'The relationship between the numbers in the question and the answer is:' -> '  \n\n          '

'To get the answer, take the two inputs and multiply them. The answer is the product of the two numbers.<|endoftext|>Q: What is the difference between a ...'